In [6]:
pip install pandas sqlalchemy pyodbc


Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from sqlalchemy import create_engine
import urllib.parse
import pyodbc
import os


In [9]:
# --------------------- CONFIGURATION ---------------------
csv_file_path = "netflix_titles.csv"  # Full path to your CSV file (use raw string with r)
server = '.'                                   # Local server (use '.' or 'DESKTOP-844MAL1')
database = 'master'                            # Change to your database name if needed
table_name = '[dbo].[netflix_raw]'                         # Your target table name

In [10]:
# Connection string (Windows Authentication - recommended for local)
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=' + server + ';'
    r'DATABASE=' + database + ';'
    r'Trusted_Connection=yes;'
)

In [11]:
# --------------------- LOAD CSV ---------------------
print("Reading CSV file...")
df = pd.read_csv(csv_file_path)

Reading CSV file...


In [12]:
# --------------------- CONNECT TO SQL SERVER ---------------------
print("\nConnecting to SQL Server...")
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
print("Connected successfully!")


Connecting to SQL Server...
Connected successfully!


In [14]:
# --------------------- INSERT DATA ---------------------
print("\nInserting data into SQL Server table...")

# Fill NaN with None (pyodbc understands None as NULL)
df = df.where(pd.notnull(df), None)

# Prepare INSERT statement
columns = ', '.join(df.columns)
placeholders = ', '.join(['?' for _ in df.columns])
insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

# Enable fast_executemany for better performance and compatibility
cursor = conn.cursor()
cursor.fast_executemany = True

try:
    # Convert DataFrame to list of tuples
    values = [tuple(row) for row in df.itertuples(index=False, name=None)]
    
    # Bulk insert all rows at once (much faster and safer than row-by-row)
    cursor.executemany(insert_query, values)
    
    conn.commit()
    print(f"Data loaded successfully! {len(df)} rows inserted into {table_name}.")

except Exception as e:
    conn.rollback()  # Undo on error
    print(f"Error during insert: {e}")
    print("Common fixes:")
    print("  - Check column data types match between CSV and table")
    print("  - Ensure no duplicate primary keys")
    print("  - Check for strings longer than VARCHAR length")
    print("  - Verify date formats are compatible")

finally:
    cursor.close()
    conn.close()
    print("Connection closed.")


Inserting data into SQL Server table...
Data loaded successfully! 8807 rows inserted into [dbo].[netflix_raw].
Connection closed.


In [15]:
df.head()
df[df.show_id=='s5023']
max(df.description.dropna().str.len())
df.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64